<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/mrc_LSTM_baseline0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import Libraries, setting Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
tf.__version__

'2.2.0'

In [0]:
import tensorflow as tf
import pickle
from tensorflow.keras import layers
from tensorflow.keras import preprocessing
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
import pprint
from tensorflow.keras.layers import Bidirectional,LSTM,Dense,Dropout,BatchNormalization,Flatten,Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import concatenate


In [0]:
# we will store the params as we go along in this object
params = {}
project_path = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/Squad2.0/TrainingDataset/"
model_path = "/content/drive/My Drive/AIML-MRC-Capstone/models/"

# Objective - LSTM Baseline 0 

*   **Inputs: A question q = {q1, ..., qQ} of length Q and a context paragraph p = {p1, ..., pP } of length P.**
*   **Output: An answer span {as, ae} where as is the index of the first answer token in p, ae is the index of the last answer token in p, 0 <= as, ae >= m, and as >= ae.** 



## 2 Load Squad Data - Cleaned and curated (output of preprocessing step)

### 2.1 Load Data

In [4]:
squad_df = pd.read_csv(project_path+'squad_data_final.csv')
squad_df.drop('Unnamed: 0',axis=1,inplace=True)
squad_df.head(2)

,title,context,question,id,answer_start,answer,plausible_answer_start,plausible_answer,is_impossible,clean_context,clean_question,clean_answer,answer_len,answer_end,answer_span,answer_word_span
0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,269,in the late 1990s,NaN,NaN,False,beyonc giselle knowlescarter bijnse beeyonsay ...,when did beyonce start becoming popular,in the late 1990s,17,286,"(269, 286)","(-1, -1)"
1,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,207,singing and dancing,NaN,NaN,False,beyonc giselle knowlescarter bijnse beeyonsay ...,what areas did beyonce compete in when she was...,singing and dancing,19,226,"(207, 226)","(21, 23)"


### 2.2 Create Train, Validation and Test data

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.utils import resample,shuffle

# train = resample(train)
# train = shuffle(train,n_samples =50000)

train,test = train_test_split(squad_df,test_size = 0.2)
train,val = train_test_split(train,test_size=0.25)

print(train.shape)
print(val.shape)
print(test.shape)

(78183, 16)
(26061, 16)
(26062, 16)


In [6]:
train["answer_word_span"] = train["answer_word_span"].apply(lambda x :eval(x))
test["answer_word_span"] = test["answer_word_span"].apply(lambda x :eval(x))
val["answer_word_span"] = val["answer_word_span"].apply(lambda x :eval(x))
train.info()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


<class 'pandas.core.frame.DataFrame'>
Int64Index: 78183 entries, 16265 to 53443
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   title                   78183 non-null  object 
 1   context                 78183 non-null  object 
 2   question                78183 non-null  object 
 3   id                      78183 non-null  object 
 4   answer_start            78183 non-null  int64  
 5   answer                  51906 non-null  object 
 6   plausible_answer_start  26276 non-null  float64
 7   plausible_answer        26276 non-null  object 
 8   is_impossible           78183 non-null  bool   
 9   clean_context           78183 non-null  object 
 10  clean_question          78183 non-null  object 
 11  clean_answer            78183 non-null  object 
 12  answer_len              78183 non-null  int64  
 13  answer_end              78183 non-null  int64  
 14  answer_span             78183 non-

### 2.3 Build Tokenizer

In [7]:
from tqdm import tqdm
params['tokenizer_num_words'] = 80000
tokenizer = preprocessing.text.Tokenizer(num_words=params['tokenizer_num_words'])

# NOTE: tokenizer is been made out of original dataset
for text in tqdm([squad_df['clean_context'], squad_df['clean_question']]):  
  tokenizer.fit_on_texts(text.values)

# total tokenizer words
params['vocab_size'] = len(tokenizer.word_index)

### SAVE TOKENIZERS
with open(model_path + "tokenizer.pkl","wb") as f:
    pickle.dump(tokenizer,f)

100%|██████████| 2/2 [00:07<00:00,  3.82s/it]


### 2.4 Update parameters

In [8]:
# From the EDA and historgrams we can conclude that - 
# 99% percentile of context word length = 285
# 99% percentile or question word lengt = 20
context_length = 285
question_length = 20
params['train_shape'] = train.shape
params['val_shape'] = val.shape
params['test_shape'] = test.shape
params['context_length_99'] = context_length # initialize with a high percentile
params['question_length_99'] = question_length # initialize with a high percentile
params['embedding_size'] = 100
params['rnn_units'] = 256
params['context_pad_seq'] = 'pre'
params['question_pad_seq'] = 'pre'

pprint.pprint(params)

{'context_length_99': 285,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': (26062, 16),
 'tokenizer_num_words': 80000,
 'train_shape': (78183, 16),
 'val_shape': (26061, 16),
 'vocab_size': 100850}


## 3 Vectorization / Encoding

#### 3.1 Integer Sequence of Context and Question 

In [0]:
train_clean_context_sequence = tokenizer.texts_to_sequences(train["clean_context"].values)
test_clean_context_sequence = tokenizer.texts_to_sequences(test["clean_context"].values)
val_clean_context_sequence = tokenizer.texts_to_sequences(val["clean_context"].values)


train_clean_question_sequence = tokenizer.texts_to_sequences(train["clean_question"].values)
test_clean_question_sequence = tokenizer.texts_to_sequences(test["clean_question"].values)
val_clean_question_sequence = tokenizer.texts_to_sequences(val["clean_question"].values)


#### 3.2 Find Max Sequence length of Context and Question

In [10]:
# max length of context
params['context_max_length'] = max(max(len(txt) for txt in train_clean_context_sequence),
                                  max(len(txt) for txt in test_clean_context_sequence),
                                  max(len(txt) for txt in val_clean_context_sequence))

params['question_max_length'] = max(max(len(txt) for txt in train_clean_question_sequence),
                                  max(len(txt) for txt in test_clean_question_sequence),
                                  max(len(txt) for txt in val_clean_question_sequence))


pprint.pprint(params)

{'context_length_99': 285,
 'context_max_length': 426,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_max_length': 40,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': (26062, 16),
 'tokenizer_num_words': 80000,
 'train_shape': (78183, 16),
 'val_shape': (26061, 16),
 'vocab_size': 100850}


#### 3.3 Padding of the sequences

In [11]:
train_context_sequence = preprocessing.sequence.pad_sequences(train_clean_context_sequence,maxlen=params['context_max_length'])

print("Max context Sequence length is {}".format(train_context_sequence.shape[1]))

test_context_sequence = preprocessing.sequence.pad_sequences(test_clean_context_sequence,maxlen=params['context_max_length'])
val_context_sequence = preprocessing.sequence.pad_sequences(val_clean_context_sequence,maxlen=params['context_max_length'])

print(train_context_sequence.shape)
print(test_context_sequence.shape)
print(val_context_sequence.shape)

Max context Sequence length is 426
(78183, 426)
(26062, 426)
(26061, 426)


In [12]:
train_question_sequence = preprocessing.sequence.pad_sequences(train_clean_question_sequence,maxlen=params['question_max_length'])
print("Max Question Sequence length is {}".format(train_question_sequence.shape[1]))
test_question_sequence = preprocessing.sequence.pad_sequences(test_clean_question_sequence,maxlen=params['question_max_length'])
val_question_sequence = preprocessing.sequence.pad_sequences(val_clean_question_sequence,maxlen=params['question_max_length'])

print(train_question_sequence.shape)
print(test_question_sequence.shape)
print(val_question_sequence.shape)


Max Question Sequence length is 40
(78183, 40)
(26062, 40)
(26061, 40)


#### 3.4 Create Answer Sequence 

Encode y_trues as big array consisting of ans_start + ans_end. This has to be used in loss function as well. We will use the answer_word_span feature

**y_true = answer_start + answer_end**

In [0]:
# for train data
y_train = []
span_ofr = 0;
params['train_span_outofrange'] = 0
params['test_span_outofrange'] = 0
params['val_span_outofrange'] = 0

for i in range(len(train)):    
    s = np.zeros(params['context_max_length'],dtype = "int")
    e = np.zeros(params['context_max_length'],dtype = "int")
    start, end = train["answer_word_span"].iloc[i]
    # if(start < params['context_length'] and end < params['context_length']):
    s[start] = 1
    e[end] = 1
    # else:
    #   span_ofr = span_ofr + 1
    #   print(start,end)
    y_train.append(np.concatenate((s,e)))    

params['train_span_outofrange'] = span_ofr
span_ofr = 0;

# for test data
y_test = []
for i in range(len(test)):    
    s = np.zeros(params['context_max_length'],dtype = "int")
    e = np.zeros(params['context_max_length'],dtype = "int")        
    start,end = test["answer_word_span"].iloc[i]    
    s[start] = 1
    e[end] = 1
    y_test.append(np.concatenate((s,e)))

params['test_span_outofrange'] = span_ofr
span_ofr = 0;
                
# for val data
y_val = []
for i in range(len(val)):
    s = np.zeros(params['context_max_length'],dtype = "int")
    e = np.zeros(params['context_max_length'],dtype = "int")        
    start,end = val["answer_word_span"].iloc[i]    
    s[start] = 1
    e[end] = 1      
    y_val.append(np.concatenate((s,e)))

params['val_span_outofrange'] = span_ofr    

In [14]:
print(len(y_train),len(y_train[0]))
print(len(y_test),len(y_test[0]))
print(len(y_val),len(y_val[0]))

78183 852
26062 852
26061 852


In [15]:
pprint.pprint(params)

{'context_length_99': 285,
 'context_max_length': 426,
 'context_pad_seq': 'pre',
 'embedding_size': 100,
 'question_length_99': 20,
 'question_max_length': 40,
 'question_pad_seq': 'pre',
 'rnn_units': 256,
 'test_shape': (26062, 16),
 'test_span_outofrange': 0,
 'tokenizer_num_words': 80000,
 'train_shape': (78183, 16),
 'train_span_outofrange': 0,
 'val_shape': (26061, 16),
 'val_span_outofrange': 0,
 'vocab_size': 100850}


In [0]:
# From the EDA and historgrams we can conclude that - 
# 99% percentile of context word length = 285
# 99% percentile or question word lengt = 20
max_context_seq_length = 285
max_question_seq_length = 20

In [0]:
print(squad_df['clean_context'][2000])
print(context_sequence[3000])
print(squad_df['clean_question'][2000])
print(questions_sequence[2000])

october 21 2008 apple reported 14 21 total revenue fiscal quarter 4 year 2008 came ipods september 9 2009 keynote presentation apple event phil schiller announced total cumulative sales ipods exceeded 220 million continual decline ipod sales since 2009 surprising trend apple corporation apple cfo peter oppenheimer explained june 2009 expect traditional mp3 players decline time cannibalize ipod touch iphone since 2009 companys ipod sales continually decreased every financial quarter 2013 new model introduced onto market
[384, 511, 58838, 520, 504, 34, 7655, 7507, 1434, 2740, 3028, 8, 22, 1434, 5566, 48288, 1070, 2828, 4793, 266, 1211, 9403, 2828, 5566, 7837, 3016, 1636, 4027, 697, 1922, 5566, 252, 2271, 4793, 27639, 99, 578, 5016, 142, 1434, 680, 5683, 142, 5566, 7, 58839, 1562, 11769, 4284, 17681, 255, 58840, 1050, 3760, 5209, 24179, 1084, 2740, 3028, 235, 1376, 115, 6740, 179, 1578, 374, 4793, 27639, 578, 8588, 44, 3439, 19, 455, 2820, 574, 193, 578, 87, 13, 11280, 60, 3366, 636]
who 

## 4 Model

**Implements a baseline 0 in Deep Learning based approach per our project synopsis. This baseline model uses the following layers **
0.   Input layer
1.   Embedding Layer
2.   List LSTM
3.   a custom Bilinear Similarity layer 
4.   Prediction Layer
5.   Output layer 



### 4.2 Building Model


**For Questions**

In [24]:
# question embedding
q_input = layers.Input(shape=(params['question_max_length'],),name="QUESTION_INPUT")
q_emb = layers.Embedding(input_dim=params['vocab_size']+1,
                  output_dim=params['embedding_size'],
                  name="QUESTION_EMBEDDING")(q_input)

# encoder 
q_output = layers.LSTM(units=params['rnn_units'], 
                     name='QUESTION_LSTM')(q_emb)
# q_cont = BiLSTM(UNITS)(q_emb)

# question context
# q_cont,_ = Attention()(q_cont) 
print(q_output.shape)

(None, 256)


**For Context**

In [28]:
c_input = layers.Input(shape=(params['context_max_length'],),name="CONTEXT_INPUT")

# context embedding
c_emb = layers.Embedding(input_dim=params['vocab_size']+1,
                  output_dim=params['embedding_size'],
                  name="CONTEXT_EMBEDDING")(c_input)



# exact_match
# ex_ = Input(shape=(CON_LEN,2))

# # pos tags
# pos_ = Input(shape=(CON_LEN,len(tag_to_num)+1))

# # term frequency
# term_ = Input(shape=(CON_LEN,1)) 

# concatenate input
# concat = concatenate([c_emb,ex_,pos_,term_])

c_output = layers.LSTM(params['rnn_units'],
                 name='CONTEXT_LSTM',return_sequences=True)(c_emb)

print("final output to bilinear ",c_output.shape)

final output to bilinear  (None, 426, 256)


**Bilinear Term**

In [36]:
# bilinear term ####
print("Question context shape ",q_output.shape)
print("final o/p of context ",c_output.shape)

################ start prediction ######################
start = layers.Dense(params['rnn_units'])(q_output)

# ading time_slice to question (batch_size,1,hidden)
# shape (64,128) --> (64,1,128)
hidden_start_time_axis = tf.expand_dims(start, -1)

# squeeze remooves time slice we added before
# final shape = (batch_size,decoder_timesteps)
start_ = tf.squeeze(tf.matmul(c_output,hidden_start_time_axis),-1)
    
start_ = tf.nn.softmax(start_,axis = 1)
    
################ end prediction ######################
end = layers.Dense(params['rnn_units'])(q_output)

hidden_end_time_axis = tf.expand_dims(end, -1)

# squeeze remooves time slice we added before
# final shape = (batch_size,decoder_timesteps)
end_ = tf.squeeze(tf.matmul(c_output,hidden_end_time_axis),-1)
end_ = tf.nn.softmax(end_,axis=1)

prob_token_span = tf.concat((start_,end_),axis = 1)
print("Logits shape ",prob.shape)

####### Prediction ### 
token_span = 20
start_prob = prob[:,:params['context_max_length']]
end_prob = prob[:,params['context_max_length']:]

# do the outer product
outer = tf.matmul(tf.expand_dims(start_prob, axis=2),tf.expand_dims(end_prob, axis=1))

outer = tf.linalg.band_part(outer, 0, token_span)

#print(outer.shape)

# start_position will have shape of (batch_size,)
start_position = tf.reduce_max(outer, axis=2)
#end position will have shape of (batch_size,)
end_position = tf.reduce_max(outer, axis=1)

y_probab = tf.concat([start_position,end_position],axis=1)


# logits = BilinearSimilarity(UNITS)(q_cont,c_)
# Y_prob = Prediction()(logits)
# print("Logits shape ",logits.shape)

Question context shape  (None, 256)
final o/p of context  (None, 426, 256)
Logits shape  (None, 852)


In [37]:
model = Model(inputs = [q_input,c_input],outputs =y_probab)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
QUESTION_INPUT (InputLayer)     [(None, 40)]         0                                            
__________________________________________________________________________________________________
QUESTION_EMBEDDING (Embedding)  (None, 40, 100)      10085100    QUESTION_INPUT[0][0]             
__________________________________________________________________________________________________
CONTEXT_INPUT (InputLayer)      [(None, 426)]        0                                            
__________________________________________________________________________________________________
QUESTION_LSTM (LSTM)            (None, 256)          365568      QUESTION_EMBEDDING[0][0]         
______________________________________________________________________________________________